In [ ]:
"""
    Pappus' creativity is due to him (implicitly) distinguishing between numerical identity and conceptual identity
    in step 1 and applying this distinction to the triangle ABC to take advantage of Euclid I.4.
"""

!python --version

## install and import packages
# !pip install nbimporter # to import notebooks
# !pip install rdflib

## import modules
import copy
# import glob
import json
# import os
# import rdflib
import re
import requests
import typing # to add advanced type hinting
import urllib

## import custom modules
import functions
import sparql_classes
import sparql_queries

# !pwd

# PROOFS OF EUCLID I.5

In [ ]:
proof_step = "<https://www.foom.com/pappus_proofPappus#00000000000000000150>"
selected_datastore:str = "/datastores/foom_pappus/sparql"
functions.find_conceptual_space_before_proof_step(proof_step, selected_datastore)

In [ ]:
# proof_iri = "<https://www.foom.com/pappus_proofAristotle#00000000000000000000>"
# selected_datastore:str = "/datastores/foom_aristotle/sparql"
# proof_steps: list = functions.find_proof_steps(proof_iri, selected_datastore)
# proof_steps

In [ ]:
proof_iri = "<https://www.foom.com/pappus_proofAristotle#00000000000000000000>"
# selected_datastore:str = "/datastores/foom_aristotle/sparql"
selected_datastore:str = "/datastores/foom_pappus/sparql"
proof_steps: list = functions.find_proof_steps(proof_iri, selected_datastore)

# start_iri: str = "<http://www.foom.com/mathematical_concepts#00000000000000000227>"
start_iri: str = "<https://www.foom.com/pappus_proofAristotle#00000000000000000001>"
# selected_datastore:str = "/datastores/foom_aristotle/sparql"

top_properties: set = {
    "<http://www.foom.com/core#00000000000000000173>", # has proof-related object property
    "<http://www.foom.com/mathematical_concepts#00000000000000000153>" # has conceptual relation
}


paths = functions.organize_nodes_paths(start_iri, top_properties, selected_datastore)
# paths

#  CHECK: IT SEEMS THAT THE LAST VERSION RETURNS TOO FEW RESULTS

conceptual_space = functions.find_conceptual_space_of_proof(
    proof_iri,
    top_properties,
    selected_datastore
)
# conceptual_space

In [ ]:
# TO DO
# 0. extreact predicate paths (for now I only extract the path of nodes)
# 1. sum conceptual space for concepts
# 2. sum conceptual space for predicates

In [ ]:
top_properties: set = {
    "<http://www.foom.com/core#00000000000000000173>", # has proof-related object property
    "<http://www.foom.com/mathematical_concepts#00000000000000000153>" # has conceptual relation
}
# selected_datastore:str = "/datastores/foom_aristotle/sparql"
# selected_datastore:str = "/datastores/foom_euclid/sparql"
selected_datastore:str = "/datastores/foom_pappus/sparql"

conceptual_space = functions.find_conceptual_space_of_proofs(
    "<https://www.foom.com/pappus_proofPappus#00000000000000000131>",
    top_properties,
    selected_datastore
)

In [ ]:
conceptual_space